In [73]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler

pd.set_option('max_columns', 100)

### Let's grab data from [basketball reference](https://www.basketball-reference.com/leagues/NBA_2019_per_game.html)

In [74]:
url = 'https://www.basketball-reference.com/leagues/NBA_2020_per_game.html'

### How can we specify just the table using beautiful soup? 

In [75]:
res = requests.get(url)
soup = BeautifulSoup(res.content)

In [76]:
table = soup.find('tbody')

thead = soup.find('thead')

headers =  [col.text for col in thead.find_all('th')]

## chopping of RK
headers = headers[1:]

In [77]:
## scraper

dcts = []
for row in table.find_all('tr', {'class':'full_table'}):
    dct = {}
    for header, datum in zip(headers, row.find_all('td')):
        dct[header] = datum.text
        
    dcts.append(dct)

### Set up the dataframe

In [78]:
df = pd.DataFrame(dcts)
df = df[headers]

In [79]:
## sort this to only the 5 major positions . .index gives us a list
positions = list(df['Pos'].value_counts()[0:5].index)

In [80]:
df = df.loc[df['Pos'].isin(positions)]

In [81]:
df.loc[df['Pos'].map(lambda x: len(x) < 3)]

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Steven Adams,C,26,OKC,48,48,27.0,4.4,7.6,.585,0.0,0.0,.000,4.4,7.5,.588,.585,1.8,3.1,.592,3.3,5.9,9.3,2.6,0.7,1.1,1.5,2.0,10.7
1,Bam Adebayo,PF,22,MIA,54,54,34.5,6.2,10.7,.579,0.0,0.2,.091,6.2,10.5,.588,.580,3.4,5.1,.681,2.6,7.8,10.4,4.9,1.2,1.2,2.7,2.4,15.8
2,LaMarcus Aldridge,C,34,SAS,50,50,33.2,7.4,14.9,.501,1.2,2.9,.407,6.3,12.0,.523,.540,3.0,3.6,.830,2.0,5.6,7.6,2.4,0.6,1.6,1.4,2.4,19.1
3,Nickeil Alexander-Walker,SG,21,NOP,41,0,12.2,1.9,5.5,.339,1.0,2.9,.342,0.9,2.7,.336,.427,0.4,0.7,.607,0.2,1.8,2.0,1.8,0.3,0.2,1.0,1.1,5.1
4,Grayson Allen,SG,24,MEM,30,0,16.6,2.6,5.9,.449,1.1,3.0,.363,1.5,2.8,.541,.543,1.0,1.2,.857,0.2,2.0,2.2,1.4,0.2,0.0,0.8,1.2,7.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,Thaddeus Young,PF,31,CHI,54,10,24.0,4.0,9.0,.441,1.1,3.3,.339,2.9,5.7,.500,.503,0.6,1.1,.586,1.4,3.3,4.7,1.7,1.2,0.3,1.6,2.0,9.7
502,Trae Young,PG,21,ATL,50,50,35.2,9.2,20.7,.443,3.5,9.4,.369,5.7,11.3,.505,.527,7.8,9.2,.852,0.6,3.8,4.4,9.2,1.2,0.1,4.9,1.7,29.7
503,Cody Zeller,C,27,CHO,50,31,23.1,4.3,8.5,.511,0.3,1.4,.243,4.0,7.1,.564,.531,2.0,2.9,.688,2.8,4.3,7.1,1.5,0.7,0.5,1.3,2.5,11.0
504,Ante Žižić,C,23,CLE,16,0,9.9,1.8,3.1,.571,0.0,0.0,,1.8,3.1,.571,.571,0.8,0.9,.867,0.9,2.6,3.4,0.2,0.4,0.3,0.3,1.3,4.3


### Set up our Xs and ys

In [82]:
### can we make our X dataframe be everything passed Minutes Played? 
X = df.loc[:, 'MP':]
y = df['Pos']

In [83]:
## Any data cleaning needed for our X? 
X = X.replace('', 0)

### Train test split 

In [84]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78741)

### Scale data

In [85]:
ss = StandardScaler()
Z_train = ss.fit_transform(X_train)
Z_test = ss.transform(X_test)

### Train KNN

In [86]:
from sklearn.metrics import accuracy_score

In [88]:
## instantiate the knn
knn = KNeighborsClassifier()
# fit the Knn
knn.fit(Z_train, y_train)
## we are accurately predicting on 54% of the test set
y_preds = knn.predict(Z_test)

accuracy_score(y_test, y_preds)

0.44

In [94]:
# accuracy score manually done
(y_preds == y_test).sum() / len(y_test)

0.44

In [95]:
y_preds

array(['PG', 'PG', 'C', 'SG', 'PG', 'PG', 'C', 'PF', 'SF', 'C', 'PF',
       'SG', 'PG', 'PF', 'SF', 'C', 'C', 'SG', 'SG', 'PG', 'SG', 'C',
       'PG', 'SG', 'SF', 'SG', 'PG', 'PG', 'C', 'C', 'SF', 'PG', 'PG',
       'SG', 'C', 'C', 'PF', 'C', 'PG', 'C', 'PG', 'SG', 'SG', 'SG', 'SF',
       'C', 'PG', 'SG', 'PF', 'C', 'C', 'C', 'SG', 'C', 'C', 'SG', 'SF',
       'SF', 'C', 'SG', 'SF', 'PG', 'PF', 'PG', 'PF', 'PF', 'C', 'PG',
       'SF', 'SG', 'C', 'C', 'PG', 'SG', 'C', 'SG', 'C', 'PF', 'C', 'PF',
       'PF', 'C', 'PG', 'C', 'SF', 'SF', 'SG', 'PF', 'PF', 'SF', 'SF',
       'PF', 'C', 'SG', 'PF', 'C', 'PG', 'C', 'SG', 'PG', 'SG', 'SF',
       'SG', 'PG', 'PF', 'SF', 'PF', 'SG', 'SG', 'PG', 'PF', 'SF', 'SF',
       'SG', 'SF', 'SG', 'PG', 'SF', 'PG', 'SF', 'SG', 'PF', 'PG', 'PF',
       'SG'], dtype=object)

In [96]:
# Baseline 
y_test.value_counts(normalize=True) # if we guessed SG for every player than we're right 28% of the time

SG    0.280
PF    0.216
SF    0.216
C     0.152
PG    0.136
Name: Pos, dtype: float64

### Create confusion matrix 

In [97]:
## look at the potential classes - create y preds 


In [98]:
## make a confusion matrix

conmat = confusion_matrix(y_test, y_preds)
# put confusion matrix in a dataframe
pd.DataFrame(conmat, columns=['Pred: '+ cls for cls in knn.classes_],\
            index=['True: '+ cls for cls in knn.classes_ ])


,Pred: C,Pred: PF,Pred: PG,Pred: SF,Pred: SG
True: C,15,4,0,0,0
True: PF,10,5,1,5,6
True: PG,0,1,14,0,2
True: SF,5,6,4,6,6
True: SG,0,4,7,9,15


### Use` .kneighbors ` to grab the matrix of nearest points - can we make this into a function?

In [101]:
## create a scaled DF of all players 
sc = StandardScaler()
X_sc = sc.fit_transform(X)
X_sc_df = pd.DataFrame(X_sc, columns=X.columns)

In [107]:
X_sc_df.index = df['Player'].values # took player values from original df to create the index for our scaled X df

In [109]:
knn_final = KNeighborsClassifier()

In [112]:
knn_final.fit(X_sc_df, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [122]:
knn_final.kneighbors(steven_adams) # 1st array is distance from steven adams to other players. 2nd is indicies of neighbors

(array([[0.        , 1.760296  , 2.06556059, 2.30551103, 2.54888202]]),
 array([[  0,   5, 144, 258, 254]]))

In [129]:
def player_comparer(name):

    player = X_sc_df[X_sc_df.index == name]
    return X_sc_df.iloc[knn_final.kneighbors(player, n_neighbors=10)[1][0]]

player_comparer('LeBron James')

,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
LeBron James,1.607954,2.747015,2.565771,0.415486,1.343358,1.561916,0.387947,2.729640,2.600538,0.398312,0.359561,1.639193,2.061448,0.010369,0.215628,2.062647,1.615511,4.896149,1.660248,-0.001899,3.386259,-0.007651,2.506074
Kyrie Irving,1.401186,2.918190,2.810347,0.326882,2.145978,1.959868,0.711919,2.623594,2.686734,0.167438,0.375232,2.337970,1.890949,0.962233,0.343673,0.675216,0.611080,2.475092,1.899064,0.235418,1.744493,1.151652,2.873726
Kawhi Leonard,1.318479,2.704222,2.708440,0.230224,1.343358,1.340832,0.612744,2.676617,2.974055,0.011259,0.163670,3.316259,2.857111,0.840718,0.215628,1.857102,1.460984,1.869827,2.854328,0.710051,2.213569,0.224209,2.827770
Luka Dončić,1.390848,2.704222,2.728822,0.222169,2.260638,2.844205,0.242490,2.305454,1.997165,0.554492,0.304711,3.945159,4.164271,0.326307,0.727808,2.730669,2.272255,3.740645,0.943800,-0.476532,3.620797,0.919791,3.103509
Nikola Jokić,1.359832,2.062316,1.709753,0.649077,0.311417,0.500712,0.282160,2.411500,2.112093,0.554492,0.469259,1.429559,1.379452,0.508579,2.008260,2.576510,2.542678,2.750212,1.421432,0.710051,2.213569,1.615373,1.832044
Russell Westbrook,1.721677,3.132159,3.197593,0.206060,0.082097,0.589145,-0.319503,3.896153,4.123337,0.099534,-0.110576,2.687359,2.743445,0.407317,0.983899,1.857102,1.692775,2.915284,2.376696,-0.001899,3.855335,2.195025,2.843088
De'Aaron Fox,1.256448,1.677172,1.668990,0.246334,0.082097,0.235411,0.116868,2.040337,2.198289,0.099534,0.014794,2.198215,2.743445,-0.013934,-0.168507,0.366898,0.224761,2.805236,2.137880,0.472735,2.565376,1.383513,1.694174
Bradley Beal,1.669985,2.918190,3.054924,0.133566,1.801998,2.313602,0.235878,2.729640,2.801662,0.208181,0.108821,3.665648,3.482274,0.642245,0.215628,0.315512,0.302025,2.365043,0.943800,-0.001899,2.448107,0.572000,3.134147
Jrue Holiday,1.576939,1.891141,2.096999,0.028853,1.228698,1.385049,0.401170,1.775221,2.054629,-0.063436,0.006958,0.521148,0.640622,0.144035,0.471718,0.315512,0.417921,2.475092,2.615512,1.184685,2.096300,0.456070,1.648217
Brandon Ingram,1.525247,2.276284,2.239669,0.286608,1.801998,1.606133,0.751589,1.987314,2.112093,0.106324,0.351725,2.757237,2.459279,0.719204,0.087583,1.394625,1.074664,1.264563,0.943800,0.710051,2.213569,1.267582,2.490755


In [124]:
## getting this to work for a player  